In [1]:
import geopandas as gpd
import xarray as xr
import numpy as np
import rioxarray as rxr
import os
from math import e
import matplotlib.pyplot as plt

In [2]:
IDir = r'/Volumes/PhD/imagery/masters/output/dNBR'
ODir = r'/Volumes/PhD/imagery/masters/output/ROC/MSM1_ROC/'

shp = r'/Volumes/PhD/imagery/masters/ROC/ROC_basins.shp'
shp = gpd.read_file(shp)
x = shp['BASIN_ID']
y = shp['RI']
RI_dict = {
    'BASIN_ID': x,
    'RI': y
}


In [ ]:
def MSM1(dnbr,
          save=False):
    
    # Coefficients for Southern California
    b = -3.63
    b_1 = 0.41
    b_2 = np.array([0.67])
    b_3 = 0.7
    
    # File naming
    
    dnbr_flat = dnbr.values.flatten()
    pixarr = np.empty((dnbr_flat.shape[0]), dtype= float)
    pixarr[:] = np.nan
    x = np.where(np.isnan(dnbr_flat) == False)
        
    # Getting the data from dnbr_flat
    dnbrdata = dnbr_flat[x]
    print(np.mean(dnbrdata))
    # Pulling tif metadata
    metadata = dnbr.attrs
    x_1 = float(metadata.get('X1'))
    x_3 = float(metadata.get('X3'))
    s = np.multiply(dnbrdata, b_2)
    
    # s1 multiplication assumes when function is called that val is defined globally in code.
    
    # Per pixel analysis, probability stored in empty list p1
    R = [3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0]
   #R = RI['RI']
    for val in R:
        s1 = np.multiply(s, val)
        p1 = []
        for i in s1:
            lnx =  b + (b_1 * x_1 * val) + i + (b_3 * x_3 * val)
            prob = (e ** lnx) / (1.0 + e ** lnx)
            p1.append(prob)
        
        # Append probability array, flatten data, reshape into image
        prob_arr = np.array(p1)
        p_final = prob_arr.flatten()
        pixarr[x] = p_final
        pixarrImg = pixarr.reshape((dnbr.shape[1], dnbr.shape[2]))
        out_dnbr = xr.Dataset()
        out_dnbr = xr.DataArray(pixarrImg, dims = ('y', 'x'),
                                coords = {'x': dnbr.coords['x'],
                                        'y': dnbr.coords['y']})
        out_dnbr = out_dnbr.where(~np.isnan(out_dnbr), np.nan)
        
        if save == True:
            outname = f"{basin}_MSM1_{int(val*4)}mmhr.tif"
            out_dnbr.rio.to_raster(os.path.join(ODir, outname), driver='GTIFF')
    

In [5]:
def MSM1_ROC(dnbr, RI,
          save=False):
    
    # Coefficients for Southern California
    b = -3.63
    b_1 = 0.41
    b_2 = np.array([0.67])
    b_3 = 0.7
    
    # File naming
    
    dnbr_flat = dnbr.values.flatten()
    pixarr = np.empty((dnbr_flat.shape[0]), dtype= float)
    pixarr[:] = np.nan
    x = np.where(np.isnan(dnbr_flat) == False)
        
    # Getting the data from dnbr_flat
    dnbrdata = dnbr_flat[x]
    print('Got dNBR')
    # Pulling tif metadata
    metadata = dnbr.attrs
    x_1 = float(metadata.get('X1'))
    x_3 = float(metadata.get('X3'))
    s = np.multiply(dnbrdata, b_2)
    print('calculated S')
    # s1 multiplication assumes when function is called that val is defined globally in code.
    
    # Per pixel analysis, probability stored in empty list p1
    
    for basin_id, R in RI.items():
        # Ensure R value exists
        if R is None:
            print(f"RI value missing for Basin ID {basin_id}")
            continue  
        print(f'{R} RI')
        s1 = np.multiply(s, R)
        p1 = []
        for i in s1:
            lnx =  b + (b_1 * x_1 * R) + i + (b_3 * x_3 * R)
            prob = (e ** lnx) / (1.0 + e ** lnx)
            p1.append(prob)
        
        # Append probability array, flatten data, reshape into image
        prob_arr = np.array(p1)
        p_final = prob_arr.flatten()
        pixarr[x] = p_final
        pixarrImg = pixarr.reshape((dnbr.shape[1], dnbr.shape[2]))
        out_dnbr = xr.Dataset()
        out_dnbr = xr.DataArray(pixarrImg, dims = ('y', 'x'),
                                coords = {'x': dnbr.coords['x'],
                                        'y': dnbr.coords['y']})
        out_dnbr = out_dnbr.where(~np.isnan(out_dnbr), np.nan)
        
        if save == True:
            outname = f"{basin}_MSM1_{int(val*4)}mmhr.tif"
            out_dnbr.rio.to_raster(os.path.join(ODir, outname), driver='GTIFF')

In [6]:
for filename in os.listdir(IDir):
    file = os.path.join(IDir, filename)
    basin = filename.split('_')
    basin = basin[1]
    print(basin)
    image = rxr.open_rasterio(file)
    MSM1_ROC(image, RI_dict, True)
    

    


10009
Got dNBR
calculated S
0      17499
1       3115
2       3589
3       5683
4       5920
       ...  
732    15065
733    35852
734    36553
735    36955
736    37056
Name: BASIN_ID, Length: 737, dtype: int64 RI


ValueError: operands could not be broadcast together with shapes (75,) (737,) 

In [8]:
for filename in os.listdir(IDir):
    file = os.path.join(IDir, filename)
    basin = filename.split('_')
    basin = basin[1]
    print(basin)

10009
10048
10058
10060
10104
10107
10109
10155
10194
10225
10242
10245
10270
10295
10346
10397
10416
10420
10421
10430
10437
10504
10505
10548
10567
10579
1057
10584
10589
1062
10650
10657
10660
10701
10718
10720
10746
10797
10807
10808
10810
10814
10828
10865
10867
10869
10890
10923
10936
11057
11060
11061
11071
11084
11092
11112
11123
11136
11171
11176
1117
11182
11186
11202
11205
11209
11217
11222
11240
11249
11265
11270
11273
1128
1130
11326
11335
1136
11375
1140
11419
11440
11441
11460
11469
11514
11522
11537
11539
11540
11556
11588
11590
11602
11610
11611
11641
11644
11647
11650
11668
11671
11680
11682
11685
11690
11724
11730
11733
11747
1174
11750
11760
11789
11797
11810
11812
11813
11829
11838
11845
11857
11868
11896
11898
1189
11918
11928
11931
11945
11953
11955
11971
11986
1200
12042
12049
12053
12057
12080
12136
12171
12205
12221
12247
12261
12288
12313
12341
12356
12357
12381
123
12417
12451
12492
12495
12518
12524
12527
1252
12571
12616
12729
12733
1277
12804
12816
12830
